# Setup for experiments

In [69]:
# Imports
import io
import boto3
import pandas as pd
import pyarrow.parquet as pq
import pyarrow as pa
from cryptography.fernet import Fernet
from io import BytesIO
from botocore.exceptions import NoCredentialsError, ClientError
from dotenv import load_dotenv
import snowflake.connector
import os

In [70]:
# Load environment variables
load_dotenv()

True

In [71]:
BUCKET_NAME = "rher-s3-test-bucket"
RAW_DATA_FILE_NAME = "employees.parquet"
SALARY_ENCRYPTED_FILE_NAME = "sample_sensitive_data_encrypted.parquet"
COLUMN_TO_ENCRYPT = "Salary" 
FERNET_KEY = os.getenv("FERNET_KEY")
ADMIN_KEY = os.getenv("ADMIN_KEY")

## Utility Functions

In [72]:
def list_s3_objects(bucket_name):
    try:
        s3 = boto3.client("s3")
        response = s3.list_objects_v2(Bucket=bucket_name)
        if 'Contents' not in response:
            print(f"No objects found in bucket: {bucket_name}")
            return []
        print(f"Objects in bucket '{bucket_name}':")
        for obj in response['Contents']:
            print(f" - {obj['Key']} (LastModified: {obj['LastModified']}, Size: {obj['Size']} bytes)")
        return [obj['Key'] for obj in response['Contents']]

    except NoCredentialsError:
        print("AWS credentials not found. Please configure them first.")
        return []
    except ClientError as e:
        print(f" AWS Client Error: {e}")
        return []
    except Exception as e:
        print(f" Unexpected error: {e}")
        return []

def read_s3_parquet(bucket_name, file_name):
    """Read a Parquet file from S3"""
    s3 = boto3.client("s3")
    response = s3.get_object(Bucket=bucket_name, Key=file_name)
    data = BytesIO(response['Body'].read())
    df = pd.read_parquet(data, engine="pyarrow")
    print(f"\n successfully loaded '{file_name}' into DataFrame.")
    return df

def encrypt_column_in_parquet(bucket_name, input_file, output_file_location, column_to_encrypt, fernet_key):
    """Encrypt a specific column in a Parquet file and upload the result to S3."""
    fernet = Fernet(fernet_key)
    s3 = boto3.client("s3")
    buffer = io.BytesIO()
    s3.download_fileobj(bucket_name, input_file, buffer)
    buffer.seek(0)
    table = pq.read_table(buffer)
    df = table.to_pandas()
    print(f"Encrypting column: {column_to_encrypt}")
    if column_to_encrypt in df.columns:
        df[column_to_encrypt] = df[column_to_encrypt].astype(str).apply(
            lambda x: fernet.encrypt(x.encode()).decode()
        )
    else:
        raise ValueError(f"Column '{column_to_encrypt}' not found in Parquet file")

    output_buffer = io.BytesIO()
    pq.write_table(pa.Table.from_pandas(df), output_buffer)
    output_buffer.seek(0)
    print("Uploading encrypted Parquet file to S3...")
    s3.upload_fileobj(output_buffer, bucket_name, output_file_location)
    print(f"Encrypted file uploaded to s3://{bucket_name}/{output_file_location}")


## Encrypt salary column in file

In [73]:
encrypt_column_in_parquet(BUCKET_NAME, RAW_DATA_FILE_NAME, SALARY_ENCRYPTED_FILE_NAME, COLUMN_TO_ENCRYPT, FERNET_KEY)

Encrypting column: Salary
Uploading encrypted Parquet file to S3...
Encrypted file uploaded to s3://rher-s3-test-bucket/sample_sensitive_data_encrypted.parquet


# Experiment 1 – Show Encrypted Data Cannot Be Read

**Goal:** Demonstrate that encrypted data stored in an AWS S3 bucket cannot be read
without the appropriate decryption key / permissions.

We will:

1. List objects in the test S3 bucket.
2. Attempt to read a Parquet file with and *encrypted* column
3. Attempt to read the same encrypted data via Snowflake external stage.


## List all .parquet fiels in bucket

In [74]:
all_keys = list_s3_objects(BUCKET_NAME)

all_parquet_files = [k for k in all_keys if k.endswith(".parquet")]
print("\nParquet objects found:")
for k in all_parquet_files:
    print(" -", k)

encrypted_candidates = [k for k in all_parquet_files if "encrypted" in k.lower()]

print("\nEncrypted candidates:")
for k in encrypted_candidates:
    print(" -", k)


Objects in bucket 'rher-s3-test-bucket':
 - employees.parquet (LastModified: 2025-11-20 14:17:33+00:00, Size: 4796 bytes)
 - sample_sensitive_data.parquet (LastModified: 2025-11-03 11:16:22+00:00, Size: 2152 bytes)
 - sample_sensitive_data_encrypted.parquet (LastModified: 2025-11-24 10:15:53+00:00, Size: 6599 bytes)

Parquet objects found:
 - employees.parquet
 - sample_sensitive_data.parquet
 - sample_sensitive_data_encrypted.parquet

Encrypted candidates:
 - sample_sensitive_data_encrypted.parquet


In [75]:
raw_data_df = read_s3_parquet(BUCKET_NAME, all_parquet_files[0])
encrypted_data_df = read_s3_parquet(BUCKET_NAME, encrypted_candidates[0])
print(f"DataFrame shape: {raw_data_df.shape}")
print(f"DataFrame shape: {encrypted_data_df.shape}")


 successfully loaded 'employees.parquet' into DataFrame.

 successfully loaded 'sample_sensitive_data_encrypted.parquet' into DataFrame.
DataFrame shape: (10, 6)
DataFrame shape: (10, 6)


In [76]:
raw_data_df.head()

,ID,Name,Email,Department,Salary,Password
0,1,Alice,alice@example.com,HR,55000,2o1e*OBZWW^7Kd
1,2,Bob,bob@example.com,Engineering,72000,GSxZX$zD5Qlf8^
2,3,Charlie,charlie@example.com,Marketing,63000,02QChWYlypvP#i
3,4,David,david@example.com,Finance,80000,m5AKDGR88&T*A8
4,5,Eva,eva@example.com,Engineering,75000,Yt%hUAUT2RX5$A


In [77]:
encrypted_data_df.head()

,ID,Name,Email,Department,Salary,Password
0,1,Alice,alice@example.com,HR,gAAAAABpJDBYCwAO0ady0MA8TwQrFXI1S-MqkiaVanFJIw...,2o1e*OBZWW^7Kd
1,2,Bob,bob@example.com,Engineering,gAAAAABpJDBYvNwep0MntUZYjAmQyN64unEp3S_iGMaoQQ...,GSxZX$zD5Qlf8^
2,3,Charlie,charlie@example.com,Marketing,gAAAAABpJDBYdIt-0wWm_5EE6nIwZtaDZtSbFZJvdMtuQi...,02QChWYlypvP#i
3,4,David,david@example.com,Finance,gAAAAABpJDBYU_F07pqy4Jh05jnOWfW3SDZ9nJj3dvf6iL...,m5AKDGR88&T*A8
4,5,Eva,eva@example.com,Engineering,gAAAAABpJDBYy-G28qez6mqwO4swBgjrdWRrkUYzukJUHW...,Yt%hUAUT2RX5$A


## Try to run with snowflake integration

In [78]:
SNOWFLAKE_ACCOUNT   = os.getenv("SNOWFLAKE_ACCOUNT")
SNOWFLAKE_USER      = os.getenv("SNOWFLAKE_USER")
SNOWFLAKE_PASSWORD  = os.getenv("SNOWFLAKE_PASSWORD")
SNOWFLAKE_ROLE      = os.getenv("SNOWFLAKE_ROLE")
SNOWFLAKE_WAREHOUSE = os.getenv("SNOWFLAKE_WAREHOUSE")
SNOWFLAKE_DATABASE  = os.getenv("SNOWFLAKE_DATABASE")
SNOWFLAKE_SCHEMA    = os.getenv("SNOWFLAKE_SCHEMA")

STAGE_NAME = "s3_test_stage"          
EXT_TABLE  = "ENGINEERINGS_SALARY_ENCRYPTED" 

snowflake_conn = snowflake.connector.connect(
    account=SNOWFLAKE_ACCOUNT,
    user=SNOWFLAKE_USER,
    password=SNOWFLAKE_PASSWORD,
    passcode=776945, ## UPDATE THIS!!
    role=SNOWFLAKE_ROLE,
    warehouse=SNOWFLAKE_WAREHOUSE,
    database=SNOWFLAKE_DATABASE,
    schema=SNOWFLAKE_SCHEMA,
)

DatabaseError: 250001 (08001): Failed to connect to DB: XBB95219.snowflakecomputing.com:443. TOTP Invalid. You have 3 attempts remaining before the user will be blocked from using authenticator apps for a few minutes.

In [ ]:
print("=== Experiment 1 (Snowflake) – Attempt to query encrypted data ===")

try:
    with snowflake_conn.cursor() as cur:
        sql = f"SELECT * FROM {EXT_TABLE} LIMIT 10"
        print("Running:", sql)
        cur.execute(sql)
        rows = cur.fetchall()
        print("\nQuery returned rows:")
        for r in rows:
            print(r)

        if not rows:
            print("\nRESULT: No rows returned. Check if the external table could not "
                  "access or parse the encrypted S3 objects.")
        else:
            print("\nRESULT: Rows returned. Inspect if data is readable vs encrypted.")
except Exception as e:
    print("\nSnowflake query failed – this is expected if encryption / KMS permissions prevent access to the S3 objects.")
    print("Error:", e)
finally:
    snowflake_conn.close()


=== Experiment 1 (Snowflake) – Attempt to query encrypted data ===

Snowflake query failed – this is expected if encryption / KMS permissions prevent access to the S3 objects.
Error: name 'snowflake_conn' is not defined


NameError: name 'snowflake_conn' is not defined

# Experiment 2 – Simulate Key Management Service + RBAC

In [ ]:

ENGINEERING = "RBAC_IN_DATA_LAKES_ROLE_READ_ONLY"
HR    = "RBAC_IN_DATA_LAKES_ROLE_SECURE_ANALYST"
ADMIN   = "RBAC_IN_DATA_LAKES_ROLE_ADMIN"

def decrypt_salary(encrypted_column):
    """Real decrypt (KMS primitive). Only this function knows the key."""
    decrypted_bytes = Fernet(FERNET_KEY).decrypt(encrypted_column.encode())
    return int(decrypted_bytes.decode())

def simulate_kms_get_key(role):
    if role == HR:
        return FERNET_KEY  # or a wrapped key
    elif role == ADMIN:
        return FERNET_KEY  # or a wrapped key
    else:
        return None
    
def get_parquet_file_from_s3(bucket_name, file_name):
    """Read a Parquet file from S3"""
    s3 = boto3.client("s3")
    response = s3.get_object(Bucket=bucket_name, Key=file_name)
    data = BytesIO(response['Body'].read())
    df = pd.read_parquet(data, engine="pyarrow")
    print(f"\n successfully loaded '{file_name}' into DataFrame.")
    return df

def decrypt_salary_with_key(key, file, encrypted_column):
    if key is None:
        print("No key provided, cannot decrypt.")
        return file
    fernet = Fernet(key)
    df = file.copy()
    df[encrypted_column] = df[encrypted_column].apply(
        lambda s: int(fernet.decrypt(s.encode()).decode())
    )
    return df


def simulate_kms(df, encrypted_column, role):
    if role == HR:
        df = df.copy()
        df[encrypted_column] = df[encrypted_column].apply(
            lambda s: decrypt_salary(s)
        )
        return df
    else:
        return df
    
    

## Test with Read-only Role

In [ ]:
key = simulate_kms_get_key(HR)
file = get_parquet_file_from_s3(BUCKET_NAME, SALARY_ENCRYPTED_FILE_NAME)
decrypt_salary_with_key(key, file, "salary")

key = simulate_kms_get_key(ENGINEERING)
file = get_parquet_file_from_s3(BUCKET_NAME, SALARY_ENCRYPTED_FILE_NAME)
decrypt_salary_with_key(key, file, "salary")

#simulate_kms(encrypted_data_df, "salary", USER).head()




 successfully loaded 'sample_sensitive_data_encrypted.parquet' into DataFrame.

 successfully loaded 'sample_sensitive_data_encrypted.parquet' into DataFrame.
No key provided, cannot decrypt.


,id,name,email,department,salary
0,1,Alice,alice@example.com,HR,gAAAAABpJC-fXe5XHhSrCOA3cQ0aoH_f2bPyeXnjK2oGSK...
1,2,Bob,bob@example.com,Engineering,gAAAAABpJC-fsBD7ycUd8qdy2-NLWZkQKG-mM97jmUPJBc...
2,3,Charlie,charlie@example.com,Marketing,gAAAAABpJC-fKLvRx6v6ZUw2Q4x9cQ-8FaV8OjhrSlt4UM...
3,4,David,david@example.com,Finance,gAAAAABpJC-fWC3ebopY4nP06kDGHEs7Bt2umLLHVs2BQl...
4,5,Eva,eva@example.com,Engineering,gAAAAABpJC-fSxXg2LHShHW508WZspMWj-6dqgyuHEcBwc...
5,6,Frank,frank@example.com,HR,gAAAAABpJC-fo4DYy1wsxxMVBOtyBmB0ML9I_AMupN9OeO...
6,7,Grace,grace@example.com,Sales,gAAAAABpJC-fTErSGlykIAgTu-R-958z3vt6IsHkrt6wlN...
7,8,Hannah,hannah@example.com,Finance,gAAAAABpJC-fcHapUxpTy8zUSH8J4OLUAgf2jMVqBqm0g-...
8,9,Ian,ian@example.com,Marketing,gAAAAABpJC-fCu0lEvQPLaROIu2iwYbZgqEdw2g15oitPO...
9,10,Julia,julia@example.com,Sales,gAAAAABpJC-fMvkGSlOBsPMBJAZ5dJNXeZ7bBFwH5W3pf1...


## Test with Secure Analyst Role

In [ ]:
simulate_kms(encrypted_data_df, "salary", HR).head()

,id,name,email,department,salary
0,1,Alice,alice@example.com,HR,55000
1,2,Bob,bob@example.com,Engineering,72000
2,3,Charlie,charlie@example.com,Marketing,63000
3,4,David,david@example.com,Finance,80000
4,5,Eva,eva@example.com,Engineering,75000


# Experiment 3 – Simulate Data Acces Layer (DAL)

Experiment simulating Data Acess Layer by:

- Creating a dummy class accessed by: `dal://...`
- The class should:
  - Uniformly enforces RBAC + Key Management
  - Decrypts Parquet columns based on role
  - Returns the parquet files with columns either decrypted/encrypted

We simulate what a DAL *would* look like in Python,
and then show why that doesn't qualify as a real, shared DAL.


In [ ]:
class DataAccessLayerClient:
    """
    Hypothetical DAL client

    In the ideal world, both Snowflake and Python would effectively
    call this logic whenever they read data from S3.
    """
    def __init__(self, base_url):
        self.base_url = base_url
    
    def read_parquet_simulate(self, bucket_path, file_name, column, role):
        """
        Simulates reading a Parquet file with DAL logic. That is, it simulates the above behaviour:
        - authenticate the user
        - check RBAC/KMS policy
        - fetch Parquet from underlying bucket and file name
        - decrypt protected columns if allowed
        - return a DataFrame or bytes of a Parquet file
        """
        ### AUTHENTICATION GOES HERE - NOT IMPLEMENTED ###
        df = read_s3_parquet(bucket_path, file_name)
        
        if simulate_kms_get_key(role):
            return simulate_kms(df, column, role)
        return "kms access denied"

In [ ]:
dal = DataAccessLayerClient(base_url="dal://rbac-sensitive/")

In [ ]:
print("Raw dataframe read directly from S3:")
display(encrypted_data_df.head())

Raw dataframe read directly from S3:


,id,name,email,department,salary
0,1,Alice,alice@example.com,HR,gAAAAABpHxOERc7yGSfVABnbcyia3Uqv9T8pcinSIZp1Su...
1,2,Bob,bob@example.com,Engineering,gAAAAABpHxOEIOXESgNTdNnZS8EhMODXl_cD1BPc5SIASx...
2,3,Charlie,charlie@example.com,Marketing,gAAAAABpHxOESCc0X4fLW2zmd-cYV5r6IyLEawkx9GcRYw...
3,4,David,david@example.com,Finance,gAAAAABpHxOEG-cDWt3ygqYmeIw9m9nCE8hFQF0mQEU_2T...
4,5,Eva,eva@example.com,Engineering,gAAAAABpHxOEPJQ9xgXuUa1ru-kSYZf0z1Zd3xQJNubSSu...


In [ ]:
print("As read_only role:")
df_ro = dal.read_parquet_simulate(bucket_path = BUCKET_NAME, file_name = SALARY_ENCRYPTED_FILE_NAME, column="salary", role=ENGINEERING)
if isinstance(df_ro, str):
    print(df_ro)
else:
    display(df_ro.head())

As read_only role:

 successfully loaded 'sample_sensitive_data_encrypted.parquet' into DataFrame.
kms access denied


In [ ]:
print("As secure_analyst (salary decrypted):")
df_sa = dal.read_parquet_simulate(bucket_path = BUCKET_NAME, file_name = SALARY_ENCRYPTED_FILE_NAME, column="salary", role=HR)
display(df_sa.head())

As secure_analyst (salary decrypted):

 successfully loaded 'sample_sensitive_data_encrypted.parquet' into DataFrame.


,id,name,email,department,salary
0,1,Alice,alice@example.com,HR,55000
1,2,Bob,bob@example.com,Engineering,72000
2,3,Charlie,charlie@example.com,Marketing,63000
3,4,David,david@example.com,Finance,80000
4,5,Eva,eva@example.com,Engineering,75000
